# CORINE Landover Preprocessing Notebook (Raster)

Create a python envionment with the Version 3.10 and activate it i.E:
```
conda create -n corine python=3.10
conda activate corine
conda install -c conda-forge rasterio geopandas shapely gdal
```


load the border file using geopandas and check its CRS

## Python Imports

In [11]:
import os
import rasterio
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [12]:
os.environ['PROJ_LIB'] = r"C:\Users\LeonardoS\miniconda3\envs\corine\Library\share\proj"

## File Paths

In [ ]:
# define vector data paths for swiss shape
border_gpkg_path = "../../../data/base/swissBOUNDARIES3D/swissBOUNDARIES3D_1_5_LV95_LN02.gpkg"

# define raster data paths for corine 
input_raster_path = "../../../data/preprocessing/corine/2018/U2018_CLC2018_V2020_20u1.tif"
output_raster_path = "../../../data/analysis/corine/2018/U2018_CLC2018_V2020_20u1.tif"

## Define Swiss Shape to clip CORINE Raster Data

coordinate reprojection from wgs84 to lv95

In [ ]:
# read swiss shape layer from geopackage
border_gdf = gpd.read_file(border_gpkg_path, layer="tlm_landesgebiet")

add 1000m buffer around swiss shape

In [ ]:
# create a 1000m buffer around the swiss shape (without other countries in the dataset)
buffered_border_gdf = border_gdf[border_gdf["icc"] == "CH"].copy()
buffered_border_gdf["geometry"] = buffered_border_gdf.buffer(1000)

remove enclaves from swiss shape

In [16]:
def fill_all_holes(geom):
    return Polygon(geom.exterior) if geom.type == 'Polygon' else MultiPolygon([Polygon(g.exterior) for g in geom.geoms])

# fill empty areas from enlaves within the swiss shape
buffered_border_gdf["geometry"] = buffered_border_gdf.geometry.apply(fill_all_holes)

C:\Users\LeonardoS\AppData\Local\Temp\ipykernel_1292\3789408810.py:2: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  return Polygon(geom.exterior) if geom.type == 'Polygon' else MultiPolygon([Polygon(g.exterior) for g in geom.geoms])


## Clip CORINE Raster Data to Swiss Shape

coordinate reprojection from wgs84 to lv95

In [ ]:
corine_raster_2056_path = "../../../data/preprocessing/corine/temp_epsg2056.tif"

with rasterio.open(input_raster_path) as src:
    
    # calculate transformation matrix
    transform, width, height = calculate_default_transform(
        src.crs, "EPSG:2056", src.width, src.height, *src.bounds
    )

    # copy and transform metadata from source raster
    kwargs = src.meta.copy()
    kwargs.update({
        "crs": "EPSG:2056",
        "transform": transform,
        "width": width,
        "height": height
    })
    
    # create output raster
    with rasterio.open(corine_raster_2056_path, "w", **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs="EPSG:2056",
                resampling=Resampling.nearest
            )

clip raster to swiss shape

In [18]:
# create list from geodataframe (use union)
buffered_border_mask= [buffered_border_gdf.union_all()]

# cut raster to the swiss shape and copy metadata from source raster
with rasterio.open(corine_raster_2056_path) as src:
    clipped_array, clipped_transform = mask(
        dataset=src,
        shapes=buffered_border_mask,
        crop=True,
        nodata=0
    )
    clipped_meta = src.meta.copy()

# update metadata for destination raster
clipped_meta.update({
    "driver": "GTiff",
    "height": clipped_array.shape[1],
    "width": clipped_array.shape[2],
    "transform": clipped_transform,
})

os.remove(corine_raster_2056_path)

save raster with swiss shape

In [10]:
# save cliped raster with accurate metadata
with rasterio.open(output_raster_path, "w", **clipped_meta) as dest:
    dest.write(clipped_array)